In [1]:
""" Notebook Information """
# References: https://www.geeksforgeeks.org/pyspark-join-types-join-two-dataframes/
# Workbook Title: PySpark Joins Tutorial - PySpark and SQL syntax

' Notebook Information '

In [2]:
""" Importing libraries """
import pandas as pd 
import numpy as np 
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext 
from pyspark.sql import SparkSession

In [3]:
""" Instantiate a SparkContext """
sc = SparkContext.getOrCreate()

## Print the Spark version
print(sc.version)

3.0.3


In [4]:
""" Creating a SparkSession """ 
spark = SparkSession.builder.appName('JoinsTutorial').getOrCreate()

In [5]:
""" creatring employees table """
# list  of employee data
data1 = [["1", "sravan", "company 1"],
        ["2", "ojaswi", "company 1"], 
        ["3", "rohith", "company 2"],
        ["4", "sridevi", "company 1"], 
        ["5", "bobby", "company 1"]]
  
# specify column names
columns = ['ID', 'NAME', 'Company']
  
# creating a df1 from the lists of data
df1 = spark.createDataFrame(data1, columns)

In [6]:
""" creating department table """
# list  of employee data
data2 = [["1", "45000", "IT"],
         ["2", "145000", "Manager"],
         ["6", "45000", "HR"],
         ["5", "34000", "Sales"]]
  
# specify column names
columns = ['ID', 'salary', 'department']
  
# creating a df1 from the lists of data
df2 = spark.createDataFrame(data2, columns)

In [7]:
""" Checking the dataframe """
# checking the dataframe  
df1.show(truncate=False)

+---+-------+---------+
|ID |NAME   |Company  |
+---+-------+---------+
|1  |sravan |company 1|
|2  |ojaswi |company 1|
|3  |rohith |company 2|
|4  |sridevi|company 1|
|5  |bobby  |company 1|
+---+-------+---------+



In [8]:
""" Checking the dataframe """
df2.show(truncate=False)

+---+------+----------+
|ID |salary|department|
+---+------+----------+
|1  |45000 |IT        |
|2  |145000|Manager   |
|6  |45000 |HR        |
|5  |34000 |Sales     |
+---+------+----------+



In [9]:
""" Example 1 - Inner Join example """
df1.join(df2, df1.ID == df2.ID, "inner").show(truncate=False)

+---+------+---------+---+------+----------+
|ID |NAME  |Company  |ID |salary|department|
+---+------+---------+---+------+----------+
|5  |bobby |company 1|5  |34000 |Sales     |
|1  |sravan|company 1|1  |45000 |IT        |
|2  |ojaswi|company 1|2  |145000|Manager   |
+---+------+---------+---+------+----------+



In [10]:
""" Example 2 - Full Join example """
df1.join(df2, df1.ID == df2.ID, "full").show(truncate=False)

+----+-------+---------+----+------+----------+
|ID  |NAME   |Company  |ID  |salary|department|
+----+-------+---------+----+------+----------+
|3   |rohith |company 2|null|null  |null      |
|5   |bobby  |company 1|5   |34000 |Sales     |
|null|null   |null     |6   |45000 |HR        |
|1   |sravan |company 1|1   |45000 |IT        |
|4   |sridevi|company 1|null|null  |null      |
|2   |ojaswi |company 1|2   |145000|Manager   |
+----+-------+---------+----+------+----------+



In [11]:
""" Example 3 - Left Join example """
df1.join(df2, df1.ID == df2.ID, "left").show(truncate=False)

+---+-------+---------+----+------+----------+
|ID |NAME   |Company  |ID  |salary|department|
+---+-------+---------+----+------+----------+
|3  |rohith |company 2|null|null  |null      |
|5  |bobby  |company 1|5   |34000 |Sales     |
|1  |sravan |company 1|1   |45000 |IT        |
|4  |sridevi|company 1|null|null  |null      |
|2  |ojaswi |company 1|2   |145000|Manager   |
+---+-------+---------+----+------+----------+



In [12]:
""" Example 4 - Right join example """
df1.join(df2, df1.ID == df2.ID, "right").show(truncate=False)

+----+------+---------+---+------+----------+
|ID  |NAME  |Company  |ID |salary|department|
+----+------+---------+---+------+----------+
|5   |bobby |company 1|5  |34000 |Sales     |
|null|null  |null     |6  |45000 |HR        |
|1   |sravan|company 1|1  |45000 |IT        |
|2   |ojaswi|company 1|2  |145000|Manager   |
+----+------+---------+---+------+----------+



In [13]:
""" Example 5 - Inner Join using SQL syntax """

## setting temporary views by creating dataframes
# creating a view for df1 named employees
df1.createOrReplaceTempView("Employees")
  
# creating a view for df2 named department
df2.createOrReplaceTempView("Department")

""" Example 5 - Inner Join based on ID """
spark.sql("SELECT * \
    FROM Employees AS E INNER JOIN \
        Department AS D ON E.ID == D.ID").show(truncate=False)

+---+------+---------+---+------+----------+
|ID |NAME  |Company  |ID |salary|department|
+---+------+---------+---+------+----------+
|5  |bobby |company 1|5  |34000 |Sales     |
|1  |sravan|company 1|1  |45000 |IT        |
|2  |ojaswi|company 1|2  |145000|Manager   |
+---+------+---------+---+------+----------+



In [14]:
""" Example 6 - Left Join based on ID """
spark.sql("SELECT * \
    FROM Employees AS E LEFT JOIN \
        Department AS D ON E.ID == D.ID").show(truncate=False)

+---+-------+---------+----+------+----------+
|ID |NAME   |Company  |ID  |salary|department|
+---+-------+---------+----+------+----------+
|3  |rohith |company 2|null|null  |null      |
|5  |bobby  |company 1|5   |34000 |Sales     |
|1  |sravan |company 1|1   |45000 |IT        |
|4  |sridevi|company 1|null|null  |null      |
|2  |ojaswi |company 1|2   |145000|Manager   |
+---+-------+---------+----+------+----------+



In [15]:
""" Example 7 - Right Join based on ID """
spark.sql("SELECT * \
    FROM Employees AS E RIGHT JOIN \
        Department AS D ON E.ID == D.ID").show(truncate=False)

+----+------+---------+---+------+----------+
|ID  |NAME  |Company  |ID |salary|department|
+----+------+---------+---+------+----------+
|5   |bobby |company 1|5  |34000 |Sales     |
|null|null  |null     |6  |45000 |HR        |
|1   |sravan|company 1|1  |45000 |IT        |
|2   |ojaswi|company 1|2  |145000|Manager   |
+----+------+---------+---+------+----------+



In [16]:
""" Example 8 - Full Join based on ID """
spark.sql("SELECT * \
    FROM Employees AS E FULL JOIN \
        Department AS D ON E.ID == D.ID").show(truncate=False)

+----+-------+---------+----+------+----------+
|ID  |NAME   |Company  |ID  |salary|department|
+----+-------+---------+----+------+----------+
|3   |rohith |company 2|null|null  |null      |
|5   |bobby  |company 1|5   |34000 |Sales     |
|null|null   |null     |6   |45000 |HR        |
|1   |sravan |company 1|1   |45000 |IT        |
|4   |sridevi|company 1|null|null  |null      |
|2   |ojaswi |company 1|2   |145000|Manager   |
+----+-------+---------+----+------+----------+

